<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>

# Laboratorio Limpieza de Datos

En este laboratorio usaremos el DataFrame de Netflix completo creado en los primeros laboratorios de Pandas. 

**Instrucciones:**

1. Lee cuidadosamente el enunciado de cada ejercicio.

2. Implementa la solución en la celda de código proporcionada.

3. Documenta todas las funciones creadas durante el ejercicio. 

4. Debes incluir después de cada gráfica la interpretación de las mismas en una celda de markdown. 

In [43]:
import pandas as pd
import numpy as np

df = pd.read_csv("../datos_entrada/netflix.csv", index_col=0)

df.head(1)

,title,Genre,Premiere,Runtime,IMDB Score,Language,originales,show_id,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Dick Johnson Is Dead,Documentary,"October 2, 2020",90.0,7.5,English,True,s1,Movie,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."


## Parte 1: Limpieza y Preparación de Datos

#### Ejercicio 1: Estandarización y limpieza de columnas

En este ejercicio, debes limpiar y estandarizar algunas columnas clave para hacerlas más manejables y consistentes en tus análisis. Específicamente, trabajarás con las columnas `date_added` y `duration` para convertirlas a un formato uniforme y estructurado.

Instrucciones:

1. **Convertir la columna `date_added`**: La columna `date_added` contiene fechas en formato de texto. Debes convertirla a un formato `datetime` que pandas pueda entender y manejar fácilmente.

2. **Limpiar la columna `duration`**: La columna `duration` tiene valores en diferentes formatos como "1 Season", "2 Seasons", "90 min", etc. Tu tarea es extraer el número (ya sea el número de temporadas o la cantidad de minutos) y crear una nueva columna llamada `duration_cleaned` con esos valores estandarizados.


**Resultado Esperado:**
Deberás obtener algo como esto:

| duration   | duration_cleaned |
|------------|-----------------|
| 1 Season   | 1               |
| 90 min     | 90              |
| 2 Seasons  | 2               |
| 45 min     | 45              |
| 3 Seasons  | 3               |

In [44]:
## Parte 1 columna date_added
# https://stackoverflow.com/questions/69726996/how-can-i-turn-a-monthyear-string-into-a-datetime-pandas luego me daba error y me daba la opcion de usar "format = 'mixed'"

df["date_added_new"] = pd.to_datetime(df["date_added"], format="mixed")

df["date_added_new"]



0      2021-09-25
1      2021-09-24
2      2021-09-24
3      2021-09-24
4      2021-09-24
          ...    
8802   2019-11-20
8803   2019-07-01
8804   2019-11-01
8805   2020-01-11
8806   2019-03-02
Name: date_added_new, Length: 8807, dtype: datetime64[ns]

In [45]:
## Parte 2 columna duration limpieza

df["duration"].unique()

# me quiero quitar todos los strings de tipo " min" y tambiend e tipo " Seasons"
# voy a usar pandas str.replace https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html

df["duration_min"] = df["duration"].str.replace(' min', '', regex=False)
df["duration_min"] = df["duration_min"].str.replace(' Seasons', '', regex=False)

df["duration_min"].unique()

df.info()





<class 'pandas.core.frame.DataFrame'>
Index: 8807 entries, 0 to 8806
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           8807 non-null   object        
 1   Genre           513 non-null    object        
 2   Premiere        513 non-null    object        
 3   Runtime         513 non-null    float64       
 4   IMDB Score      513 non-null    float64       
 5   Language        513 non-null    object        
 6   originales      8807 non-null   bool          
 7   show_id         8807 non-null   object        
 8   type            8807 non-null   object        
 9   director        6173 non-null   object        
 10  cast            7982 non-null   object        
 11  country         7976 non-null   object        
 12  date_added      8797 non-null   object        
 13  release_year    8807 non-null   int64         
 14  rating          8803 non-null   object        
 15  duration 

#### Ejercicio 2: Normalización de la columna `rating`

La columna `rating` tiene diferentes calificaciones como `PG`, `PG-13`, `R`, entre otras. Debes categorizar estas calificaciones en tres grupos:

- **'General Audience'** para calificaciones como `G`, `PG`.

- **'Teens'** para calificaciones como `PG-13`, `TV-14`.

- **'Adults'** para calificaciones como `R`, `TV-MA`.


In [46]:
# He usado el ejemplo de Raquel aqui

def rating_updater(a):
    if a == 'G' or a == 'PG' or a == 'TV-Y' or a == 'TV-G': # TV-Y no aparecen en la lista de arriba
        return 'General Audience'
    if a == 'TV-PG': # no aparecen en la lista de arriba
        return 'Parental Guidance Suggested'
    if a == 'TV-Y7' or a=='TV-Y7-FV':  # no aparece en la lista de arriba
        return 'Suitable for ages 7 and up'
    if a == 'PG-13' or a == 'TV-14':
        return 'Teens'
    if a == 'R' or a == 'TV-MA' or a == 'NC-17':
        return 'Adults'
    if a == 'UR':
        return 'Unrated by Netflix'
    else:
        return 'Unkwon rate'


df["rating_categories"] = df.apply(lambda x: rating_updater(x["rating"]), axis=1)
df[["rating_categories","rating"]]

#df["rating"].unique()

,rating_categories,rating
0,Teens,PG-13
1,Adults,TV-MA
2,Adults,TV-MA
3,Adults,TV-MA
4,Adults,TV-MA
...,...,...
8802,Adults,R
8803,Suitable for ages 7 and up,TV-Y7
8804,Adults,R
8805,General Audience,PG


#### Ejercicio 3: Creación de una columna personalizada basada en el elenco

Vamos a identificar si un actor clave como `Leonardo DiCaprio`, `Tom Hanks`, o `Morgan Freeman` aparece en el elenco.

Usa `apply` y una función lambda para crear una nueva columna llamada `has_famous_actor` que contenga `True` si alguno de estos actores está en la lista de `cast` y `False` en caso contrario.

In [ ]:


#df["cast"] = df["cast"].fillna('Unknwon')


def famous_actor_finder(cast_list):
    famous_actors = [  ## creo una lista con actores famosos
    "Leonardo DiCaprio",
    "Tom Hanks",
    "Morgan Freeman"]
    for famous in famous_actors:
        for cast in cast_list:
            if famous in str(cast):
                return True
        return False


#famous_actor_finder(df["cast"])

df["has_famous_actor"] = df.apply(lambda x: famous_actor_finder(x["cast"]), axis=1)
df[["has_famous_actor","cast"]]

## Algo me esta fallando en esta pero no acabao de verlo. 


has_famous_actor  rating  
False             TV-MA       3207
                  TV-14       2160
                  TV-PG        863
                  R            799
                  PG-13        490
                  TV-Y7        334
                  TV-Y         307
                  PG           287
                  TV-G         220
                  NR            80
                  G             41
                  TV-Y7-FV       6
                  UR             3
                  NC-17          3
                  74 min         1
                  84 min         1
                  66 min         1
Name: count, dtype: int64

    fruit
1  banana
4  orange


#### Ejercicio 4: Creación de una columna personalizada usando lógica condicional

Vamos a crear una columna llamada `is_recent` que identifique si un título fue lanzado en los últimos 5 años.

Crea una función para marcar con `True` si el título es reciente (lanzado en los últimos 5 años) y `False` si no lo es.

In [ ]:
## Me he basado en el ejemplo que vimos de Age usando la funcion cut

min_year = df["release_year"].min()
fith = df["release_year"].max()-5
max_year = df["release_year"].max()

bins_cat = [min_year,fith,max_year]

df["is_recent"] = pd.cut(x=df['release_year'], bins=bins_cat, labels=[False, True])

df["is_recent"]



0        True
1        True
2        True
3        True
4        True
        ...  
8802    False
8803     True
8804    False
8805    False
8806    False
Name: is_recent, Length: 8807, dtype: category
Categories (2, bool): [False < True]

#### Ejercicio 5: Clasificación de películas por década

En este ejercicio, tu objetivo es categorizar los años de lanzamiento de las películas o series en décadas. La columna `release_year` contiene el año de lanzamiento y debes crear una nueva columna llamada `decade` que indique la década correspondiente, como "1990s", "2000s", etc.


In [ ]:
## La parte del diccionario es de ChatGPT, el resto no.
# Crear un diccionario para las décadas
decadas = {}

# Rango de años de 1900 a 2029
for anio in range(df["release_year"].min(), df["release_year"].max()):
    # Obtener la década correspondiente (ejemplo: 1990s)
    decada = f"{anio // 10 * 10}s"
    # Añadir la década al diccionario si no existe
    if decada not in decadas:
        decadas[decada] = []
    # Agregar el año a la lista correspondiente
    decadas[decada].append(anio)

# Mostrar el diccionario
print(decadas)

## funcion que comorueba si un anio dado existe en una decada y devuelve la decada correspondiente o desconocido si no la encuentra
def asignar_decada(year):
    for decade, years in decadas.items():
        if year in years:
            return decade
    return "Unknown"  # por si hay alguna peli fuera de ese rango o sin anios, pero no lo parece

# Aplicar la función a la columna "release_year"
df["decade"] = df["release_year"].apply(asignar_decada) # apply es el metodo que va a iterar cada registro en el campo "release_year"


{'1920s': [1925, 1926, 1927, 1928, 1929], '1930s': [1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939], '1940s': [1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949], '1950s': [1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959], '1960s': [1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969], '1970s': [1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979], '1980s': [1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989], '1990s': [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999], '2000s': [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009], '2010s': [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019], '2020s': [2020]}


#### Ejercicio 6: Extracción de información

Para practicar la extracción de información:

1. **Extrae el primer actor** de la lista en la columna `cast` y crea una nueva columna llamada `first_actor`.

2. **Extrae el primer nombre del director** y guárdalo en una columna llamada `first_name_director`.


In [103]:
df['first_actor'] = df['cast'].str.split(',').str[0] ## https://saturncloud.io/blog/how-to-extract-first-and-last-words-from-strings-as-a-new-column-in-pandas/

df['first_actor']

df['first_director'] = df['director'].str.split(',').str[0] ## https://saturncloud.io/blog/how-to-extract-first-and-last-words-from-strings-as-a-new-column-in-pandas/

df['first_director']



0       Kirsten Johnson
1                   NaN
2       Julien Leclercq
3                   NaN
4                   NaN
             ...       
8802      David Fincher
8803                NaN
8804    Ruben Fleischer
8805       Peter Hewitt
8806        Mozez Singh
Name: first_director, Length: 8807, dtype: object

#### Ejercicio 7: Limpieza de la columna `cast`

La columna `cast` contiene una lista de actores separados por comas. Tu objetivo es realizar las siguientes tareas:

1. **Reemplaza los valores nulos** en la columna `cast` por "sin información".

2. **Contar el número de actores** en cada entrada y crear una nueva columna llamada `num_cast`.

3. **Normalizar los nombres**: Asegúrate de que los nombres de los actores estén en un formato consistente (por ejemplo, quitar espacios adicionales).


In [119]:
## 1. Reemplazar NaN
df["cast"].fillna("Sin Informacion", inplace=True)

df["cast"].unique()

## 2. Contar el numero de actores separados por comas  en num_cast
#df['num_cast'] = len(df['cast'].str.split(','))
#df['num_cast']

df['num_cast'] = df['cast'].apply(lambda x: len(x.split(',')))

df[['cast','num_cast']]


## 3. Quita espacios adicionales

df['cast_clean'] = df['cast'].apply(lambda x: x.replace('  ', ''))

df[['cast','cast_clean']]

,cast,cast_clean
0,Sin Informacion,Sin Informacion
1,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...","Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban..."
2,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...","Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi..."
3,Sin Informacion,Sin Informacion
4,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...","Mayur More, Jitendra Kumar, Ranjan Raj, Alam K..."
...,...,...
8802,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...","Mark Ruffalo, Jake Gyllenhaal, Robert Downey J..."
8803,Sin Informacion,Sin Informacion
8804,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...","Jesse Eisenberg, Woody Harrelson, Emma Stone, ..."
8805,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...","Tim Allen, Courteney Cox, Chevy Chase, Kate Ma..."



#### Ejercicio 8: Identificación de Directores Recurrentes

En este ejercicio, debes identificar los directores que aparecen más de una vez en el conjunto de datos. Realiza los siguientes pasos:

1. **Reemplaza los valores nulos** en la columna `director` por "sin información".

3. **Cuenta cuántas veces aparece cada director** en la columna creada en el ejercicio 6.

4. **Filtra aquellos directores que aparecen más de una vez** y crea una nueva columna llamada `recurrent_director` donde se indique "Yes" si el director aparece varias veces o "No" en caso contrario.

In [ ]:
## 1. Reemplazar NaN
df["director"].fillna("Sin Informacion", inplace=True)

df["director"].unique()

### 2. Cuenta cuantas veces aparece cada director en la columna creada en el ejercio 6

# df["times_director"] = df['first_director'].value_counts()  ## no funciona directamente

director_counts = df["first_director"].value_counts()  

print(type(director_counts))
print(director_counts)  # esto son los nombres de cada director con el numero de veces que apaecere

# Crea la columna times_director con lo anterior usando map
df["times_director"] = df["first_director"].map(director_counts)

### 3. Recurrent director

def recurrent_director_function(n):
    if n > 1:
        return "Yes"
    else:
        return "No"


df["recurrent_director"] = df["times_director"].apply(recurrent_director_function)

df[["recurrent_director", "times_director"]]


<class 'pandas.core.series.Series'>
first_director
Rajiv Chilaka    22
Raúl Campos      18
Marcus Raboy     16
Suhas Kadav      16
Jay Karas        15
                 ..
Jung Ji-woo       1
Matt D'Avella     1
Parthiban         1
Scott McAboy      1
Mozez Singh       1
Name: count, Length: 4405, dtype: int64


,recurrent_director,times_director
0,No,1.0
1,No,NaN
2,Yes,3.0
3,No,NaN
4,No,NaN
...,...,...
8802,Yes,6.0
8803,No,NaN
8804,Yes,2.0
8805,No,1.0


In [ ]:
df.to_pickle("JaviSoto_Netflix.pkl")